In [1]:
%matplotlib inline
import math as m
import numpy as np
import matplotlib.pyplot as plt
from SAMPEX_functions import read_counts as read
from SAMPEX_functions import mb_finder, iso_calculator
import pandas as pd

In [2]:
# reading in electron counts file: day ### of year ####
year = '1993'
start_day = '053'

file = 'hhrr' + year + start_day + '.txt'
t, r1, r2, r3, r4 = read(file)

In [3]:
# find microburst times and N_100, SSD1, SSD4 counts using algorithm

t_microburst, N_100_microburst, r1_microburst, r4_microburst, mb_index, MB_mask N_100, A_500 = mb_finder(t, r1, r2, r3, r4)

ValueError: too many values to unpack (expected 7)

In [ ]:
# calculation of the isotropy indices of electron counts

iso_indices, iso_indices_plot = iso_calculator(r1_microburst, r4_microburst)

In [ ]:
# reading in orbit/attitude data file: day ### of year ####

directory = 'D:\SAMPEX_Data\\'
OA_file = 'OrbAtt_' + year + start_day + '.txt'
OrbAtt_data = pd.read_csv(directory + OA_file, names = ['sec_of_day', 'GEO_Radius', 'GEO_Long', 'GEO_Lat', 'Altitude', 'L_Shell', 'MLT', 
                                         'SAA_Flag', 'Pitch', 'zenith', 'azimuth', 'Att_Flag'], sep = '\s+', header = 70)

# Augment OrbAtt data to fit counts data

L_Shell = OrbAtt_data['L_Shell'].values; sec_of_day = OrbAtt_data['sec_of_day'].values; MLT = OrbAtt_data['MLT'].values;
Pitch = OrbAtt_data['Pitch'].values; Lat = OrbAtt_data['GEO_Lat'].values; Long = OrbAtt_data['GEO_Long'].values

# trim the start of OrbAtt data to match that of the counts data

while sec_of_day[0] != t[0]:
    sec_of_day = sec_of_day[1:]; L_Shell = L_Shell[1:]; MLT = MLT[1:]; Pitch = Pitch[1:]; Lat = Lat[1:]; Long = Long[1:]

# create OrbAtt arrays with the same length as microburst arrays

t_OrbAtt = [sec_of_day[0]]; LS_OrbAtt = [L_Shell[0]]; MLT_OrbAtt = [MLT[0]];
P_OrbAtt = [Pitch[0]]; Lat_OrbAtt = [Lat[0]]; Long_OrbAtt = [Long[0]]

a = 0  # OrbAtt index
b = 0  # Counts index

while True:
    b += 1
    if b >= len(t):
        break
#    print(b)    #counter
    if a+1 < len(sec_of_day):    # this handles end conditions
        
        if t[b] - t[b-1] > 1:       # this handles breaks in the electron data (catches up the OA data index)
            while sec_of_day[a] < int(t[b]):
                a += 1

        currentsec = sec_of_day[a]
        nextsec = sec_of_day[a+1]
    
        if abs(currentsec - t[b]) >= abs(nextsec - t[b]):
            a += 1

    t_OrbAtt.append(sec_of_day[a]); LS_OrbAtt.append(L_Shell[a]); MLT_OrbAtt.append(MLT[a]); 
    P_OrbAtt.append(Pitch[a]); Lat_OrbAtt.append(Lat[a]); Long_OrbAtt.append(Long[a])
    
t_OrbAtt = np.array(t_OrbAtt); LS_OrbAtt = np.array(LS_OrbAtt); MLT_OrbAtt = np.array(MLT_OrbAtt);
P_OrbAtt = np.array(P_OrbAtt); Lat_OrbAtt = np.array(Lat_OrbAtt); Long_OrbAtt = np.array(Long_OrbAtt)

t_OrbAtt_mb = t_OrbAtt[mb_index]; LS_OrbAtt_mb = LS_OrbAtt[mb_index]; MLT_OrbAtt_mb = MLT_OrbAtt[mb_index];
P_OrbAtt_mb = P_OrbAtt[mb_index]; Lat_OrbAtt_mb = Lat_OrbAtt[mb_index]; Long_OrbAtt_mb = Long_OrbAtt[mb_index]

# 1. Iso vs Parameter correlations

In [ ]:
plt.figure(figsize = (30,15))
plt.title('Microburst Isotropy vs L-Shell', fontsize = 20)
plt.scatter(LS_OrbAtt_mb, iso_indices_plot)
plt.xlabel('L-Shell', fontsize = 15)
plt.ylabel('Microburst Isotropy [1-scale]', fontsize = 15)
plt.ylim(0, 1.1)
plt.xlim(3,7)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.grid(True)

In [ ]:
plt.figure(figsize = (30,15))
plt.title('Microburst Isotropy vs Pitch Angle', fontsize = 20)
plt.scatter(P_OrbAtt_mb, iso_indices_plot)
plt.xlabel('Spacecraft Pitch Angle (degrees)', fontsize = 15)
plt.ylabel('Microburst Isotropy [1-scale]', fontsize = 15)
plt.ylim(0, 1.1)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.grid(True)

In [ ]:
plt.figure(figsize = (30,15))
plt.title('Microburst Isotropy vs Magnetic Local Time', fontsize = 20)
plt.scatter(MLT_OrbAtt_mb, iso_indices_plot)
plt.xlabel('Magnetic Local Time', fontsize = 15)
plt.ylabel('Microburst Isotropy [1-scale]', fontsize = 15)
plt.ylim(0, 1.1)
plt.xlim(-1, 25)
plt.xticks([0, 3, 6, 9, 12, 15, 18, 21, 24])
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.grid(True)

In [ ]:
plt.figure(figsize = (18,8))
plt.title('Microburst Isotropy vs Spacecraft Latitude', fontsize = 20)
plt.scatter(Lat_OrbAtt_mb, iso_indices_plot)
plt.xlabel('Spacecraft Latitude (degrees)', fontsize = 15)
plt.ylabel('Microburst Isotropy [1-scale]', fontsize = 15)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.grid(True)

# 2. Parameter-time/location analysis

In [ ]:
# L-Shell vs time plot

plt.figure(figsize = (30,15))
plt.title('L-Shell vs Time (with microburst markers)', fontsize = 25)
plt.plot(sec_of_day, L_Shell)
plt.plot(t_microburst, LS_OrbAtt_mb, 'bo')
plt.xlabel('Time (seconds)', fontsize = 15)
plt.ylabel('L-Shell', fontsize = 15)
plt.ylim(0, 10)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.grid(True)

# Latitude vs time plot

plt.figure(figsize = (30,15))
plt.title('Latitude vs Time (with microburst markers)', fontsize = 25)
plt.plot(sec_of_day, Lat)
plt.plot(t_microburst, Lat_OrbAtt_mb, 'bo')
plt.xlabel('Time (seconds)', fontsize = 15)
plt.ylabel('Latitude', fontsize = 15)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.grid(True)

In [ ]:
# Pitch Angle vs:

# time plot

plt.figure(figsize = (30,15))
plt.title('Pitch Angle vs Time', fontsize = 25)
plt.plot(sec_of_day, Pitch)
plt.xlabel('Time (seconds)', fontsize = 15)
plt.ylabel('Pitch Angle (degrees)', fontsize = 15)
#plt.ylim(0, 20)
plt.xticks(fontsize = 15)
plt.yticks(np.linspace(0, 180, 10), fontsize = 15)
plt.grid(True)

# lat plot

plt.figure(figsize = (30,15))
plt.title('Pitch Angle vs Latitude', fontsize = 25)
plt.plot(Lat, Pitch)
plt.xlabel('Latitude (degrees)', fontsize = 15)
plt.ylabel('Pitch Angle (degrees)', fontsize = 15)
#plt.ylim(0, 20)
plt.xticks(np.linspace(-90, 90, 13), fontsize = 15)
plt.yticks(np.linspace(0, 180, 10), fontsize = 15)
plt.grid(True)

In [ ]:
# create subplots

fig, axes = plt.subplots(1, 2, figsize = (18,8))
left = axes[0]
right = axes[1]

# Position (viewed from above Earth): y vs x

t = np.linspace(0, 2*m.pi, 100)  # set of parameters
radius_E = 6375                  # approx Earth radius (km)
x_circle = radius_E * np.cos(t)  # circle x-val
y_circle = radius_E * np.sin(t)  # circle y-val

Radius = OrbAtt_data['GEO_Radius'].values
Radius = Radius[1:]

start_i = 0 
length = 1500

r = Radius[start_i:start_i+length]; theta = 90 - Lat[start_i:start_i+length];

fi = Long[start_i:start_i+length]          # longitude method
#fi = MLT[start_i:start_i+length] * 15      # MLT method

x_pos = r * np.sin(theta * m.pi/180) * np.cos(fi * m.pi/180); y_pos = r * np.sin(theta * m.pi/180) * np.sin(fi * m.pi/180);
z_pos = r * np.cos(theta * m.pi/180)

left.plot(x_circle, y_circle, 'g')
left.plot(x_pos, y_pos, color = 'k')
left.set_title('Position in Orbit (xy-plane)')
left.set_xlabel('x-position (km)')
left.set_ylabel('y-position (km)')
left.legend(['Earth Surface', 'Satellite Orbit'], loc = 'best')
left.grid(True)

right.plot(x_circle, y_circle, 'g')
right.plot(x_pos, z_pos, color = 'k')
right.set_title('Position in Orbit (xz-plane)')
right.set_xlabel('x-position (km)')
right.set_ylabel('z-position (km)')
right.legend(['Earth Surface', 'Satellite Orbit'], loc = 'best')
right.grid(True)

# 3. Parameter-frequency analysis

In [ ]:
# create subplots

fig, axes = plt.subplots(1, 2, figsize = (18,8))
left = axes[0]
right = axes[1]

# pitch angle histogram

bin_num = 18
P_mean = np.mean(P_OrbAtt)
domain = 200

left.hist(P_OrbAtt, bin_num, color = 'dodgerblue')
left.set_title('Pitch Angle Histogram')
left.set_xlabel('Pitch Angle (degrees)')
left.set_ylabel('Frequency')
left.set_xlim([0,180])
#left.set_ylim(None, 1000)
left.set_xticks(np.linspace(0, 180, 10))
left.grid(True)

# Magnetic Local Time histogram

bin_num = 25
MLT_mean = np.mean(MLT_OrbAtt)
domain = 26

right.hist(MLT_OrbAtt, bin_num, color = 'lightcoral')
right.set_title('Magnetic Local Time Histogram')
right.set_xlabel('MLT')
right.set_ylabel('Frequency')
right.set_xlim([0,24])
right.set_xticks(np.linspace(0, 24, 9))
right.grid(True)